<a href="https://colab.research.google.com/github/Just1919/Pneumonia_Detection/blob/main/P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **ROJET DETECTION DE PNEUMONIA**

In [1]:
#Installationde Tensorflow
!pip install Tensorflow

In [2]:
#Importation des packages
import os
import numpy as np
import tensorflow as tf
from glob import glob

from keras.applications.vgg16 import VGG16

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten
from keras.models import Model

### **Connexion de colab à Kaggle**

In [ ]:
#Création d'un dossier Kaggle
! mkdir ~/.kaggle/

In [4]:
#Copier kaggle.jason vers le dossier  créé
!cp '/content/drive/MyDrive/Kaggle/kaggle.json'  ~/.kaggle/

In [5]:
# donner les droits a kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
#Afficher les bases de données
! kaggle datasets list

ref                                                  title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                  Online Food Dataset                                  3KB  2024-03-02 18:50:30           9354        170  0.9411765        
syedanwarafridi/vehicle-sales-data                   Vehicle Sales Data                                  19MB  2024-02-21 20:16:17          19106        337  1.0              
julianoorlandi/spotify-top-songs-and-audio-features  Spotify Top Songs and Audio Features               526KB  2024-03-18 12:49:50           1235         24  1.0              
bhavikjikadara/student-study-performance             Student Study Performance                            9KB  2024-03-0

### **Télécharger notre dataset**

In [ ]:
! kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:23<00:00, 198MB/s]
100% 2.29G/2.29G [00:23<00:00, 104MB/s]


In [ ]:
!unzip /content/drive/MyDrive/Kaggle/chest-xray-pneumonia.zip -d pneumonia_data

In [7]:
#Importer la base de donnée
data_path="/content/drive/MyDrive/Kaggle/pneumonia_data/chest_xray"

In [8]:
#Spécifier le chemin des  datasets de  train et test
train_dir=os.path.join(data_path,"train")
test_dir=os.path.join(data_path,"test")

**Applications Keras**: https://keras.io/api/applications/

In [9]:
#Préparer le modèle de base
IMG_SHAPE=(224,224,3)
base_model=VGG16(input_shape=IMG_SHAPE,include_top=False, weights="imagenet")

58889256/58889256 [==============================] - 0s 0us/step


In [10]:
#Bloquer le modèle de base
base_model.trainable=False

In [11]:
base_model.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [12]:
#Ajouter les couches de sorties
x=Flatten()(base_model.output)
prediction=Dense(2, activation='softmax')(x)

In [13]:
#Création du modèle globale
model=Model(inputs=base_model.input,outputs=prediction)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
model.compile(
              loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

In [15]:
#Créer les photos augmantées

train_datagen=ImageDataGenerator(

                                 rescale=1/255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,)
test_datagen=ImageDataGenerator(rescale=1/255)

training_set=train_datagen.flow_from_directory(train_dir,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode="categorical")
test_set=test_datagen.flow_from_directory(test_dir,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode="categorical")




Found 1455 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [16]:
#Appliquer le modèle

model.fit(training_set,epochs=5,validation_data=test_set)

Epoch 1/5
46/46 [==============================] - 1402s 31s/step - loss: 0.0182 - accuracy: 0.9828 - val_loss: 15.3532 - val_accuracy: 0.6250
Epoch 2/5
46/46 [==============================] - 1282s 28s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 15.5053 - val_accuracy: 0.6250
Epoch 3/5
46/46 [==============================] - 1283s 28s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 15.5069 - val_accuracy: 0.6250
Epoch 4/5
46/46 [==============================] - 1285s 28s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 15.5069 - val_accuracy: 0.6250
Epoch 5/5
46/46 [==============================] - 1290s 28s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 15.5069 - val_accuracy: 0.6250


In [18]:
valid_loss,valid_accuracy=model.evaluate(test_set)

20/20 [==============================] - 394s 20s/step - loss: 15.5069 - accuracy: 0.6250


In [ ]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))